In [1]:
import cv2
import numpy as np
import pandas as pd
import time
import joblib
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\anush\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Lightweight pose model
model = YOLO("yolov8n-pose.pt")


In [5]:
def extract_features(keypoints):
 

    left_shoulder = keypoints[5]
    right_shoulder = keypoints[6]
    left_hip = keypoints[11]
    right_hip = keypoints[12]

    # Shoulder midpoint
    shoulder_mid = (left_shoulder + right_shoulder) / 2
    hip_mid = (left_hip + right_hip) / 2

    dx = hip_mid[0] - shoulder_mid[0]
    dy = hip_mid[1] - shoulder_mid[1]

    torso_angle = np.arctan2(dy, dx)

    body_height = np.linalg.norm(shoulder_mid - hip_mid)
    body_width = np.linalg.norm(left_shoulder - right_shoulder)

    ratio = body_width / (body_height + 1e-6)

    return [torso_angle, body_height, body_width, ratio]


In [12]:
import os

base_path = "C:/Users/anush/Downloads/archive (1)/laying_dataset"

print("Folder exists:", os.path.exists(base_path))
print("Files inside folder:")
print(os.listdir(base_path))

Folder exists: True
Files inside folder:
['dataset.yaml', 'example_image.png', 'images', 'labels']


In [14]:

with open("C:/Users/anush/Downloads/archive (1)/laying_dataset/dataset.yaml") as f:
    print(f.read())

path: /
train: images
val: images
kpt_shape:
- 17
- 3
names:
  0: laying
  1: standing



In [19]:
import os
print(os.listdir("C:/Users/anush/Downloads/archive (1)/laying_dataset/images"))

['laying01.png', 'laying02.png', 'laying03.png', 'laying04.png', 'laying05.png', 'laying06.png', 'laying07.png', 'laying08.png', 'laying09.png', 'laying10.png', 'laying100.png', 'laying101.png', 'laying102.png', 'laying103.png', 'laying104.png', 'laying105.png', 'laying107.png', 'laying108.png', 'laying109.png', 'laying11.png', 'laying110.png', 'laying111.png', 'laying112.png', 'laying113.png', 'laying12.png', 'laying13.png', 'laying14.png', 'laying15.png', 'laying16.png', 'laying17.png', 'laying18.png', 'laying19.png', 'laying20.png', 'laying21.png', 'laying22.png', 'laying23.png', 'laying24.png', 'laying25.png', 'laying26.png', 'laying27.png', 'laying28.png', 'laying29.png', 'laying30.png', 'laying31.png', 'laying32.png', 'laying33.png', 'laying34.png', 'laying35.png', 'laying36.png', 'laying37.png', 'laying38.png', 'laying39.png', 'laying40.png', 'laying41.png', 'laying42.png', 'laying43.png', 'laying44.png', 'laying45.png', 'laying46.png', 'laying47.png', 'laying48.png', 'laying49.

In [24]:
import os

label_path = "C:/Users/anush/Downloads/archive (1)/laying_dataset/labels"
print(os.listdir(label_path)[:5])

['laying01.txt', 'laying02.txt', 'laying03.txt', 'laying04.txt', 'laying05.txt']


In [25]:
with open(os.path.join(label_path, os.listdir(label_path)[0])) as f:
    print(f.read())
    

0 0.5850992202758789 0.5320504307746887 0.46550238132476807 0.26178836822509766 0.3903105854988098 0.4249964654445648 2 0.38966476917266846 0.41156646609306335 2 0.37620672583580017 0.4279046058654785 2 0.40826040506362915 0.4109205901622772 2 0.3767626881599426 0.4491688907146454 2 0.45758476853370667 0.43612027168273926 2 0.41166946291923523 0.480001837015152 2 0.49866148829460144 0.43800947070121765 2 0.37034258246421814 0.5169642567634583 2 0.5442894101142883 0.45122605562210083 2 0.3777313530445099 0.4694572389125824 2 0.5760704874992371 0.47892674803733826 2 0.5492944121360779 0.5114280581474304 2 0.6719107031822205 0.5360683798789978 2 0.6615118980407715 0.5517429709434509 2 0.7706350684165955 0.5821139216423035 2 0.7663392424583435 0.6153815984725952 2



In [28]:
from ultralytics import YOLO

model = YOLO("yolov8n-pose.pt")

model.train(
    data="C:/Users/anush/Downloads/archive (1)/laying_dataset/dataset.yaml",
    epochs=50,
    imgsz=640,
    batch=8,
    device="cpu"
)

Ultralytics 8.4.18  Python-3.14.2 torch-2.10.0+cpu CPU (AMD Ryzen 7 7445HS w/ Radeon 740M Graphics)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/anush/Downloads/archive (1)/laying_dataset/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train6, nbs=64, nms=False, opset=None, optimize=False, opti

ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002124CBC3F50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.0340

In [30]:
import os

base = r"C:\Users\anush\ANUSHRABA\FYRP\VitaCustos_AI\training\runs\pose"
print(os.listdir(base))

['train', 'train2', 'train3', 'train4', 'train5', 'train6']


In [ ]:

import cv2
import numpy as np
from ultralytics import YOLO
model = YOLO("yolov8n-pose.pt")


cap = cv2.VideoCapture(0)


ASPECT_RATIO_THRESHOLD = 1.2  
CONFIDENCE_THRESHOLD = 0.5

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    
    results = model(frame)

    for result in results:
        boxes = result.boxes

        for box in boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])

           
            if cls == 0 and conf > CONFIDENCE_THRESHOLD:

                x1, y1, x2, y2 = map(int, box.xyxy[0])
                width = x2 - x1
                height = y2 - y1

                aspect_ratio = width / height

                color = (0, 255, 0)

                if aspect_ratio > ASPECT_RATIO_THRESHOLD:
                    label = "FALL DETECTED"
                    color = (0, 0, 255)
                else:
                    label = "Person"

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} ({conf:.2f})",
                            (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.7,
                            color,
                            2)

    cv2.imshow("YOLOv11 Fall Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 1 person, 117.6ms
Speed: 3.0ms preprocess, 117.6ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 86.5ms
Speed: 3.7ms preprocess, 86.5ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 86.6ms
Speed: 2.3ms preprocess, 86.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 67.6ms
Speed: 1.8ms preprocess, 67.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 70.6ms
Speed: 1.9ms preprocess, 70.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 72.1ms
Speed: 1.6ms preprocess, 72.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 74.1ms
Speed: 1.8ms preprocess, 74.1ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 65.9ms
Speed: 1.6ms preprocess, 65.9ms inference, 1.0ms postprocess per image at shape (1, 3, 